In [1]:
import os
from dotenv import load_dotenv
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import (
    BasePromptTemplate,
    FewShotPromptTemplate
)
from src.grag.retrievers.text2cypher.examples import (
    text2cypher_example,
    text2cypher_example_prompt
)
from src.grag.retrievers.text2cypher.prompts import CYPHER_GENERATION_PROMPT

load_dotenv(".env")

True

In [2]:
embedder_model = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

In [3]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=text2cypher_example,
    embeddings=embedder_model,
    vectorstore_cls=InMemoryVectorStore,
    k=3
)

In [4]:
few_shot_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=text2cypher_example_prompt,
    suffix="## Current User Question:\nQuestion: {question}\nNeo4j Cypher: \n",
    # input_variables=["question"],
)

print(few_shot_prompt_template.invoke({"question": "Apa isi pasal 1 UU Nomor 11 Tahun 2005".lower()}).to_string())

Question: apa isi pasal habis pasal 15 peraturan menteri kominfo (permenkominfo) no. 10 tahun 2008
Neo4j Cypher: 
```cypher
MATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)-[:NEXT_ARTICLE]->(next_article)
WHERE r.type = 'PERMENKOMINFO' AND r.number = 10 AND r.year = 2008 AND a.number = '15'
RETURN next_article.text AS text
```


Question: apa pasal selanjutnya dari pasal 9 undang-undang / uu nomor 10 tahun 2010
Neo4j Cypher: 
```cypher
MATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)-[:NEXT_ARTICLE]->(next_article)
WHERE r.type = 'UU' AND r.number = 10 AND r.year = 2010 AND a.number = '9'
RETURN next_article.text AS text
```


Question: apa isi konten dari pasal 50 pp (peraturan pemerintah) no. 70 tahun 2015
Neo4j Cypher: 
```cypher
MATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)
WHERE r.type = 'PP' AND r.number = 70 AND r.year = 2015 AND a.number = '50'
RETURN a.text AS text
```


## Current User Question:
Question: apa isi pasal 1 uu nomor 11 tahun 2005
Neo4j Cypher: 



In [5]:
test = CYPHER_GENERATION_PROMPT.invoke({
    "question": "<<< QUESTION >>>",
    "schema": "<<< SCHEMA >>>",
    "example": "<<< EXAMPLE >>>"
}).to_string()

print(test)


## Task:
- Generate Neo4j Cypher statement to query a graph database.

## Instructions:
- Make a cypher code for user query or user questions.
- Use only the provided relationship types and properties in the schema.
- Do not use any other relationship types or properties that are not provided.

## Schema:

<<< SCHEMA >>>

## Note:
- Do not include any explanations or apologies in your responses.
- Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
- Do not include any text except the generated Cypher statement.

## Examples:
Following are some examples that you can use as a reference to create Cypher code according to user questions.

<<< EXAMPLE >>>

## Current User Question:

<<< QUESTION >>>



In [6]:
from langchain_core.runnables import RunnableLambda

few_shot_selection = RunnableLambda(
    lambda x: {
        "question": few_shot_prompt_template.invoke({
            "question": x.lower()
        }).to_string(),
        "schema": "<<< SCHEMA >>>",
        "example": "<<< EXAMPLE >>>",
        "dummy": "<<< CUMA COBA LEBIHIN PARAMETER NYA >>>"
    }
)

chain = few_shot_selection | CYPHER_GENERATION_PROMPT
print(chain.invoke("Apa yang dimaksud dengan data pribadi?").to_string())


## Task:
- Generate Neo4j Cypher statement to query a graph database.

## Instructions:
- Make a cypher code for user query or user questions.
- Use only the provided relationship types and properties in the schema.
- Do not use any other relationship types or properties that are not provided.

## Schema:

<<< SCHEMA >>>

## Note:
- Do not include any explanations or apologies in your responses.
- Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
- Do not include any text except the generated Cypher statement.

## Examples:
Following are some examples that you can use as a reference to create Cypher code according to user questions.

<<< EXAMPLE >>>

## Current User Question:

Question: apa isi konten dari pasal 50 pp (peraturan pemerintah) no. 70 tahun 2015
Neo4j Cypher: 
```cypher
MATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)
WHERE r.type = 'PP' AND r.number = 70 AND r.year = 2015 AND a.number = '50'
RETURN a.text AS text
`

In [7]:
import os
from pprint import pprint
from dotenv import load_dotenv
from langchain_neo4j import Neo4jGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from src.grag.retrievers.text2cypher.text2cypher import create_text2cypher_retriever_tool

load_dotenv(".env")

True

In [8]:
URI = os.environ["DATABASE_HOST"]
DATABASE = os.environ["DATABASE_SMALL"]
USERNAME = os.environ["DATABASE_USERNAME"]
PASSWORD = os.environ["DATABASE_PASSWORD"]

neo4j_graph = Neo4jGraph(
    url=URI,
    username=USERNAME,
    password=PASSWORD,
    database=DATABASE,
    enhanced_schema=True
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.0,
    api_key=os.environ["GOOGLE_API_KEY"]
)

In [9]:
text2cypher_retriever = create_text2cypher_retriever_tool(
    cypher_llm=llm,
    qa_llm=llm,
    neo4j_graph=neo4j_graph,
    skip_qa_llm=False,
    verbose=True
)

In [10]:
few_shot_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=text2cypher_example_prompt,
    suffix="",
    input_variables=["question"],
)

print(few_shot_prompt_template.invoke({"question": "Apa isi pasal 1 UU Nomor 11 Tahun 2005".lower()}).to_string())

Question: apa isi pasal habis pasal 15 peraturan menteri kominfo (permenkominfo) no. 10 tahun 2008
Neo4j Cypher: 
```cypher
MATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)-[:NEXT_ARTICLE]->(next_article)
WHERE r.type = 'PERMENKOMINFO' AND r.number = 10 AND r.year = 2008 AND a.number = '15'
RETURN next_article.text AS text
```


Question: apa pasal selanjutnya dari pasal 9 undang-undang / uu nomor 10 tahun 2010
Neo4j Cypher: 
```cypher
MATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)-[:NEXT_ARTICLE]->(next_article)
WHERE r.type = 'UU' AND r.number = 10 AND r.year = 2010 AND a.number = '9'
RETURN next_article.text AS text
```


Question: apa isi konten dari pasal 50 pp (peraturan pemerintah) no. 70 tahun 2015
Neo4j Cypher: 
```cypher
MATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)
WHERE r.type = 'PP' AND r.number = 70 AND r.year = 2015 AND a.number = '50'
RETURN a.text AS text
```



In [11]:
from langchain.globals import set_debug

set_debug(True)

In [12]:
from langchain_core.runnables import RunnableLambda

few_shot_selection = RunnableLambda(
    lambda x: {
        "query": x,
        "example": few_shot_prompt_template.invoke({
            "question": x.lower()
        }).to_string()
    }
)

# chain = few_shot_selection | text2cypher_retriever
input_data = few_shot_selection.invoke("Apa isi pasal 28 UU Nomor 11 tahun 2008?")
input_data

[chain/start] [chain:RunnableLambda] Entering Chain run with input:
{
  "input": "Apa isi pasal 28 UU Nomor 11 tahun 2008?"
}
[chain/start] [chain:RunnableLambda > prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "question": "apa isi pasal 28 uu nomor 11 tahun 2008?"
}
[chain/end] [chain:RunnableLambda > prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[chain/end] [chain:RunnableLambda] s] Exiting Chain run with output:
{
  "query": "Apa isi pasal 28 UU Nomor 11 tahun 2008?",
  "example": "Question: apa isi pasal habis pasal 15 peraturan menteri kominfo (permenkominfo) no. 10 tahun 2008\nNeo4j Cypher: \n```cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)-[:NEXT_ARTICLE]->(next_article)\nWHERE r.type = 'PERMENKOMINFO' AND r.number = 10 AND r.year = 2008 AND a.number = '15'\nRETURN next_article.text AS text\n```\n\n\nQuestion: apa pasal selanjutnya dari pasal 9 undang-undang / uu nomor 10 tahun 2010\nNeo4j Cypher: \n```cypher\nMATCH (r:R

{'query': 'Apa isi pasal 28 UU Nomor 11 tahun 2008?',
 'example': "Question: apa isi pasal habis pasal 15 peraturan menteri kominfo (permenkominfo) no. 10 tahun 2008\nNeo4j Cypher: \n```cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)-[:NEXT_ARTICLE]->(next_article)\nWHERE r.type = 'PERMENKOMINFO' AND r.number = 10 AND r.year = 2008 AND a.number = '15'\nRETURN next_article.text AS text\n```\n\n\nQuestion: apa pasal selanjutnya dari pasal 9 undang-undang / uu nomor 10 tahun 2010\nNeo4j Cypher: \n```cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)-[:NEXT_ARTICLE]->(next_article)\nWHERE r.type = 'UU' AND r.number = 10 AND r.year = 2010 AND a.number = '9'\nRETURN next_article.text AS text\n```\n\n\nQuestion: apa isi konten dari pasal 50 pp (peraturan pemerintah) no. 70 tahun 2015\nNeo4j Cypher: \n```cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)\nWHERE r.type = 'PP' AND r.number = 70 AND r.year = 2015 AND a.number = '50'\nRETURN a.text AS text\n```\n"}

In [13]:
text2cypher_retriever.invoke(input_data)

# Apa isi pasal 28 UU Nomor 11 tahun 2008?
# {'query': 'Apa isi pasal 28 UU Nomor 11 tahun 2008?'}


# > Entering new GraphCypherQAChainMod chain...
# {'query': 'Apa isi pasal 28 UU Nomor 11 tahun 2008?'}
# {'query': 'Apa isi pasal 28 UU Nomor 11 tahun 2008?'}

[tool/start] [tool:text2cypher] Entering Tool run with input:
"{'query': 'Apa isi pasal 28 UU Nomor 11 tahun 2008?', 'example': "Question: apa isi pasal habis pasal 15 peraturan menteri kominfo (permenkominfo) no. 10 tahun 2008\nNeo4j Cypher: \n```cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)-[:NEXT_ARTICLE]->(next_article)\nWHERE r.type = 'PERMENKOMINFO' AND r.number = 10 AND r.year = 2008 AND a.number = '15'\nRETURN next_article.text AS text\n```\n\n\nQuestion: apa pasal selanjutnya dari pasal 9 undang-undang / uu nomor 10 tahun 2010\nNeo4j Cypher: \n```cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)-[:NEXT_ARTICLE]->(next_article)\nWHERE r.type = 'UU' AND r.number = 10 AND r.year = 2010 AND a.number = '9'\nRETURN next_article.text AS text\n```\n\n\nQuestion: apa isi konten dari pasal 50 pp (peraturan pemerintah) no. 70 tahun 2015\nNeo4j Cypher: \n```cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)\nWHERE r.type = 'PP' AND r.number = 70 AND r.year = 2015 A

'### **Hasil Pembuatan Kode Cypher:**\n```cypher\nMATCH (r:Regulation {type: \'UU\', number: 11, year: 2008})\n  -[:HAS_ARTICLE]->(a:Article {number: "28"})\nRETURN a.text\n```\n\n### **Hasil Eksekusi Kode Cypher ke Database:**\nPasal 28 UU Nomor 11 Tahun 2008 berisi tentang:\n\n(1) Setiap Orang dengan sengaja dan tanpa hak menyebarkan berita bohong dan menyesatkan yang mengakibatkan kerugian konsumen dalam Transaksi Elektronik.\n(2) Setiap Orang dengan sengaja dan tanpa hak menyebarkan informasi yang ditujukan untuk menimbulkan rasa kebencian atau permusuhan individu dan/atau kelompok masyarakat tertentu berdasarkan atas suku, agama, ras, dan antargolongan (SARA).'

In [14]:
set_debug(False)
# print(chain.invoke("Apa isi pasal 28 UU Nomor 11 tahun 2008?"))